In [8]:
import os
import gensim
import jieba
import sys
sys.path.append('../')
from langconv import *
import re
from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence
import logging
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

In [12]:
cut_file =  '/python/datasource/wiki_data/data/wikireduce.txt'

In [13]:
def wiki_process(wiki_file_name,cut_file):
    space = ' '
    i = 0
    l = []        
    f = open(cut_file,'a',encoding = 'utf-8')
    wikiFile = open(wiki_file_name,'r',encoding='utf-8')
    for text in wikiFile:
        text = text.strip()
        text = str(text)
        if not text: continue
        if re.match(r'<.*>',text): continue
        temp_sentence = Converter('zh-hans').convert(text)
        temp_sentence = "".join(re.findall(r'\w+',temp_sentence)        )
        seg_list = list(jieba.cut(temp_sentence))
        for temp_term in seg_list:
            l.append(temp_term)        
        f.write(space.join(l) + '\n')
        l = []
        i = i + 1
        if (i%20000 == 0):
            print('Saved ' + str(i) + 'lines')            
    f.close()


In [14]:
def train():
    wiki_news = open(cut_file, 'r',encoding='utf-8')
    model = Word2Vec(LineSentence(wiki_news),sg=0, size=300, window=5,min_count=5, workers=4)
    model.save('D:\python\datasource\wiki_data\wiki_data.word2vec')

In [ ]:
wiki_dir = '/python/datasource/wiki_data/data/extract/AA'
cut_dir = '/python/datasource/wiki_data/data/'
pathDir = os.listdir(wiki_dir)
for f in pathDir:
    print(f)
    child = os.path.join('%s/%s'%(wiki_dir, f))
#     cut_file = os.path.join('%s/%s.txt'%(cut_dir, f))
    wiki_process(child, cut_file)
train()

In [16]:
model = Word2Vec.load("/python/datasource/wiki_data/wiki_data.word2vec")

D:\ProgramData\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import re
import jieba

from gensim.models import Word2Vec
from gensim.models.word2vec import LineSentence

import networkx as nx

D:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
def get_connect_graph_by_text_rank(tokenized_text, window=3):
    keywords_graph = nx.Graph()
    tokeners = tokenized_text.split()
    for ii, t in enumerate(tokeners):
        word_tuples = [(tokeners[connect], t) 
                       for connect in range(ii-window, ii+window+1) 
                       if connect >= 0 and connect < len(tokeners)]
        keywords_graph.add_edges_from(word_tuples)

    return keywords_graph


In [3]:
def TextRank(cut_sents):
    #connect graph
    words_graph = get_connect_graph_by_text_rank(" ".join(cut_sents[0]))

    #draw networkx
    plt.rcParams['font.sans-serif'] = ['SimHei']
    plt.figure(3, figsize=(12, 12))
    nx.draw_networkx(words_graph, font_size=12)

    #get dictory
    g_dic = {}
    node_set = set()
    for i, j in words_graph.edges():
        node_set.add(i)
        node_set.add(j)
        if i == j: continue
        if i not in g_dic : g_dic[i] = set()
        g_dic[i].add(j)

    #matrics
    node_mat = np.zeros((len(node_set), len(node_set)))
    for i, w1 in enumerate(g_dic):
        p = 1/len(g_dic[w1])
        for j, w2 in enumerate(g_dic[w1]):
            node_mat[j][i] = p
    node_mat

    #text rank
    d = 0.85
    pr = np.ones((len(node_set), 1)) 
    for i in range(10):
        print(pr.T)
        pr = 1/len(node_set) * (1-d) + d * np.dot(node_mat,pr)
    print('\n',pr.T)

In [4]:
fname = '/python/datasource/sqlResult_1558435.csv'

news = pd.read_csv(fname, encoding = 'gb18030')
len(news)

89611